<img src="NotebookAddons/blackboard-banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>SAR/Optical (NDVI) Time Series Analysis</b></font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" style="padding:5px;" width="170" align="right" /></font>

<font size="3">This notebook compares time series data of C-band Backscatter and Landsat-8 NDVI over a forested site in the Hindu Kush Himalaya region. It showcases similarities and differences between the two data sets and shows seasonal dependences of both the radar (Sentinel-1) and Landsat-8 data for this area.
    
The exercise is done in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>This notebook covers the following data analysis concepts:</b>

- How to combine SAR and optical data into consistent time series
- How to compare subset means between optical and SAR acquisitions
- Differences and similarities between the optical and SAR time series information


</font>


</font>

<hr>
<font face="Calibri" size="5" color='rgba(200,0,0,0.2)'> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


In [ ]:

%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
// alert(command)
kernel.execute(command)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
    <li> The <b><a href="https://www.pydoc.io/pypi/asf-hyp3-1.1.1/index.html" target="_blank">asf-hyp3 API</a></b> provides useful functions and scripts for accessing and processing SAR data via the Alaska Satellite Facility's Hybrid Pluggable Processing Pipeline, or HyP3 (pronounced "hype"). </li>
<li><b><a href="https://www.scipy.org/about.html" target="_blank">SciPY</a></b> is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics. </li>

</font>

<font face="Calibri" size="3"> Our first step is to <b>import them:</b> </font>

In [ ]:
%%capture
# Check Python version:
import sys
pn = sys.version_info[0]

# Importing relevant python packages
from math import ceil
import os

import pandas as pd
import numpy as np
from osgeo import gdal # for Info
from skimage import exposure # to enhance image display

import asf_notebook as asfn
asfn.jupytertheme_matplotlib_format()


# For plotting
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.patches as patches

font = {'family' : 'monospace',
          'weight' : 'bold',
          'size'   : 18}
plt.rc('font',**font)

plt.rcParams.update({'font.size': 12})


import copy

if pn == 2:
    import cStringIO #needed for the image checkboxes
elif pn == 3:
    import io
    import base64


#---------------------------------------------------------------------------------------------

def moving_window(data, radius=3, verbose=False):
    '''Applies a moving window average to a 2D array and returns the result.
    
    Parameters:
    - data: 2D array of raster values
    - radius: Radius size in pixels
    - verbose: Option to print a message whenever a pixel cannot be added to
        the window
    '''
    [nrow, ncol] = data.shape
    windowed = copy.copy(data)
    for row in range(nrow):
        for col in range(ncol):
            vals_in_window = []
            row_rng = range(row-radius,row+radius+1)
            col_rng = range(col-radius,col+radius+1)
            for y in row_rng:
                for x in col_rng:
                    try:
                        val = data[y][x]
                        vals_in_window.append(val)
                    except Exception as e:
                        if verbose:
                            print("Could not add pixel to list, " +
                                  "may be due to edge effects")
                            print(e)
            windowed[row][col] = np.mean(vals_in_window)
    return(windowed)



#---------------------------------------------------------------------------------------------

# Define a helper function for a 4 part figure with backscatter, NDVI and False Color Infrared
def ebd_plot(bandnbrs):
    fig,ax=plt.subplots(2,2,figsize=(13,13))
    # Bands for sentinel and landsat:
    # Sentinel VV
    sentinel_vv=img_handle[0].GetRasterBand(bandnbrs[0]).ReadAsArray(*subset_sentinel)
    sentinel_vv=20.*np.log10(sentinel_vv)-83 # Covert to dB
    # Sentinel VH
    sentinel_vh=img_handle[1].GetRasterBand(bandnbrs[1]).ReadAsArray(*subset_sentinel)
    sentinel_vh=20.*np.log10(sentinel_vh)-83 # Covert to dB
    # # Landsat False Color InfraRed
    r=img_handle[5].GetRasterBand(bandnbrs[2]).ReadAsArray(*subset_landsat)/10000.
    g=img_handle[4].GetRasterBand(bandnbrs[2]).ReadAsArray(*subset_landsat)/10000.
    b=img_handle[3].GetRasterBand(bandnbrs[2]).ReadAsArray(*subset_landsat)/10000.
    fcir=np.dstack((r,g,b))
    for i in range(fcir.shape[2]):
        fcir[:,:,i] = exposure.\
        equalize_hist(fcir[:,:,i],
        mask=~np.equal(fcir[:,:,i],-.9999))
    # Landsat NDVI
    landsat_ndvi=img_handle[2].GetRasterBand(bandnbrs[1]).ReadAsArray(*subset_landsat)
    mask=landsat_ndvi==-9999
    landsat_ndvi = landsat_ndvi/10000. # Scale to real NDVI value
    landsat_ndvi[mask]=np.nan
    svv = ax[0][0].imshow(sentinel_vv,cmap='jet',vmin=np.nanpercentile(sentinel_vv,5),
                   vmax=np.nanpercentile(sentinel_vv,95))
    cb = fig.colorbar(svv,ax=ax[0][0],orientation='horizontal')
    cb.ax.set_title('C-VV $\gamma^o$ [dB]')
    svh = ax[0][1].imshow(sentinel_vh,cmap='jet',vmin=np.nanpercentile(sentinel_vh,5),
                   vmax=np.nanpercentile(sentinel_vh,95))
    cb = fig.colorbar(svh,ax=ax[0][1],orientation='horizontal')
    cb.ax.set_title('C-VH $\gamma^o$ [dB]')

    nvmin=np.nanpercentile(landsat_ndvi,5)
    nvmax=np.nanpercentile(landsat_ndvi,95)
    # nvmin=-1
    # nvmax=1
    nax = ax[1][0].imshow(landsat_ndvi,cmap='jet',vmin=nvmin,
                   vmax=nvmax)
    cb = fig.colorbar(nax,ax=ax[1][0],orientation='horizontal')
    cb.ax.set_title('NDVI')

    fc= ax[1][1].imshow(fcir)
    # cb = fig.colorbar(fc,cmap=cm.gray,ax=ax[1][1],orientation='horizontal')
    # cb.ax.set_title('False Color Infrared')

    ax[0][0].axis('off')
    ax[0][1].axis('off')
    ax[1][0].axis('off')
    ax[1][1].axis('off')
    ax[0][0].set_title('Sentinel-1 C-VV {}'.format(stindex[0][bandnbrs[0]-1].date()))
    ax[0][1].set_title('Sentinel-1 C-VH {}'.format(stindex[1][bandnbrs[1]-1].date()))
    ax[1][0].set_title('Landsat-8 NDVI {}'.format(ltindex[bandnbrs[2]-1].date()))
    ax[1][1].set_title('Landsat-8 False Color IR {}'.format(ltindex[bandnbrs[2]-1].date()))
    _=fig.suptitle('Sentinel-1 Backscatter and Landsat NDVI and FC IR',size=16)
    
    

#---------------------------------------------------------------------------------------------

# Define a helper function for a 4 part figure with backscatter, RFDI, and NDVI
def RFDI_NDVI_plot(bandnbrs):
    fig,ax=plt.subplots(2,2,figsize=(13,13))
    # Bands for sentinel and landsat:
    # Sentinel VV
    sentinel_vv=img_handle[0].GetRasterBand(bandnbrs[0]).ReadAsArray(*subset_sentinel)
    sentinel_vvdB=20.*np.log10(sentinel_vv)-83 # Covert to dB
    sentinel_vv=np.power(10., sentinel_vvdB/10.) # Covert to power    
    # Sentinel VH
    sentinel_vh=img_handle[1].GetRasterBand(bandnbrs[1]).ReadAsArray(*subset_sentinel)
    sentinel_vhdB=20.*np.log10(sentinel_vh)-83 # Covert to dB
    sentinel_vh=np.power(10., sentinel_vhdB/10.) # Covert to power
    # # Sentinel-1 RFDI
    s1_rfdi = (sentinel_vv - sentinel_vh)/(sentinel_vv + sentinel_vh)
    s1_rfdi = moving_window(s1_rfdi, radius=3)
    # Landsat NDVI
    landsat_ndvi=img_handle[2].GetRasterBand(bandnbrs[1]).ReadAsArray(*subset_landsat)
    mask=landsat_ndvi==-9999
    landsat_ndvi = landsat_ndvi/10000. # Scale to real NDVI value
    landsat_ndvi[mask]=np.nan
    
    svv = ax[0][0].imshow(sentinel_vv,cmap='jet',vmin=np.nanpercentile(sentinel_vv,5),
                   vmax=np.nanpercentile(sentinel_vv,95))
    cb = fig.colorbar(svv,ax=ax[0][0],orientation='horizontal')
    cb.ax.set_title('C-VV $\gamma^o$ [dB]')
    
    svh = ax[0][1].imshow(sentinel_vh,cmap='jet',vmin=np.nanpercentile(sentinel_vh,5),
                   vmax=np.nanpercentile(sentinel_vh,95))
    cb = fig.colorbar(svh,ax=ax[0][1],orientation='horizontal')
    cb.ax.set_title('C-VH $\gamma^o$ [dB]')

    rfdimin=np.nanpercentile(s1_rfdi,5)
    rfdimax=np.nanpercentile(s1_rfdi,95)
    rfdiax = ax[1][0].imshow(s1_rfdi,cmap='jet',vmin=rfdimin,
                   vmax=rfdimax)
    cb = fig.colorbar(rfdiax,ax=ax[1][0],orientation='horizontal')
    cb.ax.set_title('Sentinel-1 RFDI')
    
    nvmin=np.nanpercentile(landsat_ndvi,5)
    nvmax=np.nanpercentile(landsat_ndvi,95)
    # nvmin=-1
    # nvmax=1
    nax = ax[1][1].imshow(landsat_ndvi,cmap='jet',vmin=nvmin,
                   vmax=nvmax)
    cb = fig.colorbar(nax,ax=ax[1][1],orientation='horizontal')
    cb.ax.set_title('NDVI')

    ax[0][0].axis('off')
    ax[0][1].axis('off')
    ax[1][0].axis('off')
    ax[1][1].axis('off')
    ax[0][0].set_title('Sentinel-1 C-VV {}'.format(stindex[0][bandnbrs[0]-1].date()))
    ax[0][1].set_title('Sentinel-1 C-VH {}'.format(stindex[1][bandnbrs[1]-1].date()))
    ax[1][0].set_title('Sentinel-1 RFDI {}'.format(stindex[0][bandnbrs[0]-1].date()))
    ax[1][1].set_title('Landsat-8 NDVI {}'.format(ltindex[bandnbrs[2]-1].date()))
    _=fig.suptitle('Sentinel-1 Backscatter, RFDI, and Landsat NDVI',size=16)


<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Data Stack</b> </font> <img style="padding:7px;" src="NotebookAddons/Nepalclimate.jpeg" width="400" align="right" /> 

<font size="3"> This notebook will be using Sentinel-1 and Landsat-8 SAR and optical time series information over the site in southern Nepal. The C-band time series includes both VV-polarized (70-image deep stack) and VH-polarized (38 images) data. The Landsat-8 stack is 53 images deep. The C-band data were acquired by the ALOS PALSAR sensor and are available to us through the services of the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility</a>. 

Nepal is an interesting site for this analysis due to the significant seasonality of precipitation that is characteristic for this region. Nepal is said to have five seasons: spring, summer, monsoon, autumn and winter. Precipitation is low in the winter (November - March) and peaks dramatically in the summer, with top rain rates in July, August, and September (see figure to the right). As SAR is sensitive to changes in soil moisture, these weather patterns have a noticeable impact on the Radar Cross Section ($\sigma$) time series information. 

</font></font>
<hr>
<br>
<font face="Calibri" size="3"> Before we download anything, <b>create a working directory for this analysis and change into it:</b> </font>

In [ ]:
path = "/home/jovyan/notebooks/SAR_Training/English/Ecosystems/data_SAR_optical_comparison"
asfn.new_directory(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

In [ ]:
s3_path = 's3://asf-jupyter-data/time_series.zip'
time_series_path = os.path.basename(s3_path)
!aws --region=us-east-1 --no-sign-request s3 cp $s3_path $time_series_path

In [ ]:
if asfn.path_exists(time_series_path):
    asfn.asf_unzip(os.getcwd(), time_series_path)
    os.remove(time_series_path)

<font face="Calibri" size="3"> The following lines set path variables needed for data processing. This step is not necessary but it saves a lot of extra typing later.<b> Define variables for the main data directory as well as for the files containing data and image information:</b></font>

In [ ]:
datadirectory=f"{path}/time_series/S32644X696260Y3052060sS1-EBD"
datefile_like='S32644X696260Y3052060sS1_D_vv_0092_mtfil.dates'
datefile_cross='S32644X696260Y3052060sS1_D_vh_0092_mtfil.dates'
imagefile_like='S32644X696260Y3052060sS1_D_vv_0092_mtfil.vrt'
imagefile_cross='S32644X696260Y3052060sS1_D_vh_0092_mtfil.vrt'
sentinel1_datefile_like=datefile_like
sentinel1_datefile_cross=datefile_cross
sentinel1_imagefile=imagefile_like
sentinel1_imagefile_cross=imagefile_cross
landsat8_ndvi='landsat/L8_142_041_NDVI.vrt'
landsat8_b3='landsat/L8_142_041_B3.vrt'
landsat8_b4='landsat/L8_142_041_B4.vrt'
landsat8_b5='landsat/L8_142_041_B5.vrt'
landsat8_datefile='landsat/L8_142_041_NDVI.dates'

<br>
<hr>
<font face="Calibri" size="5"> <b> 2. Switch to the Data Directory: </b></font>

In [ ]:
if asfn.path_exists(datadirectory):
    os.chdir(datadirectory)
print(f"current directory: {os.getcwd()}")

<br>
<hr>
<font face="Calibri" size="5"> <b> 3. Assess Image Acquisition Dates </b> </font> 

<font face="Calibri" size="3"> Before we start analyzing the available image data, we want to examine the content of our data stack. <b>First, we read the image acquisition dates for all files in the time series and create a *pandas* date index.</b> </font>

In [ ]:
print(sentinel1_datefile_like)
print(sentinel1_datefile_cross)
[sentinel1_datefile_like,sentinel1_datefile_cross]

In [ ]:
stindex=[]
for i in [sentinel1_datefile_like,sentinel1_datefile_cross]:
    sdates=open(i).readlines()
    stindex.append(pd.DatetimeIndex(sdates))
    j=1
    print('Bands and dates for',i.strip('.dates'))
    for k in stindex[-1]:
        print("{:4d} {}".format(j, k.date()),end=' ')
        j+=1
        if j%5==1: print()

In [ ]:
ldates=open(landsat8_datefile).readlines()
ltindex=pd.DatetimeIndex(ldates)
j=1
print('Bands and dates for',landsat8_ndvi)
for i in ltindex:
    print("{:4d} {}".format(j, i.date()),end=' ')
    j+=1
    if j%5==1: print()

<br>
<hr>
<font face="Calibri" size="5"> <b> 4. Projection and Georeferencing Information of the SAR and Optical Time Series Data Stacks </b> </font> 

<font face="Calibri" size="3"> For processing of the imagery in this notebook we generate a list of image handles and retrieve projection and georeferencing information. We print out the retrieved information.</font>

In [ ]:
imagelist=[sentinel1_imagefile,sentinel1_imagefile_cross,landsat8_ndvi,landsat8_b3,landsat8_b4,landsat8_b5]
geotrans=[]
proj=[]
img_handle=[]
xsize=[]
ysize=[]
bands=[]
for i in imagelist:
    img_handle.append(gdal.Open(i))
    geotrans.append(img_handle[-1].GetGeoTransform())
    proj.append(img_handle[-1].GetProjection())
    xsize.append(img_handle[-1].RasterXSize)
    ysize.append(img_handle[-1].RasterYSize)
    bands.append(img_handle[-1].RasterCount)
# for i in proj:
#     print(i)
# for i in geotrans:
#     print(i)
for i in zip(['C-VV','C-VH','NDVI','B3','B4','B5'],bands,ysize,xsize):
     print(i)

<br>
<hr>
<font face="Calibri" size="5"> <b> 5. Display SAR and NDVI  Images </b> </font> 

<font face="Calibri" size="3">First, depending on the capacity of the computer we might want to define a subset. If you are using the Open SAR Lab for processing, no subsetting is needed. If you download this notebook and run it on your own machine it might be prudent to subset the scene using the code-cell below. The example below chooses the subset in the raster extension of the Sentinel-1 Image and use the geotransformation information to extract the corresponding subset in the Landsat Image. We assume that the images have the same upper left coordinate. The we can compute the offsets and extent in the Landsat image as follows:

$x_{cal} = \frac{xres_{sentinel-1}}{xres_{landsat}}$

$y_{cal} = \frac{yres_{sentinel-1}}{yres_{landsat}}$

We can use these calibration factors to get the landsat subset as follows:

- $xoff_{landsat} = xoff_{sentinel-1} \times x_{cal}$ 
- $yoff_{landsat} = yoff_{sentinel-1} \times y_{cal}$ 
- $xsize_{landsat} = xsize_{sentinel-1} \times x_{cal}$ 
- $ysize_{landsat} = ysize_{sentinel-1} \times y_{cal}$ 

(xoffset,yoffset,xsize,ysize) </font>

In [ ]:
subset_sentinel=None
subset_sentinel=(570,40,500,500)  # Adjust or comment out if you don't want a subset
if subset_sentinel == None:
    subset_sentinel=(0,0,img_handle[0].RasterXSize,img_handle[0].RasterYSize)
    subset_landsat=(0,0,img_handle[2].RasterXSize,img_handle[2].RasterYSize)
else:
    xoff,yoff,xsize,ysize=subset_sentinel
    xcal=geotrans[0][1]/geotrans[2][1]
    ycal=geotrans[0][5]/geotrans[2][5]
    subset_landsat=(int(xoff*xcal),int(yoff*ycal),int(xsize*xcal),int(ysize*ycal))

print('Subset Sentinel-1',subset_sentinel,'\nSubset Landsat   ',subset_landsat)

<br>
<hr>
<font face="Calibri" size="5"> <b> 6. Visualize SAR and Landsat-8 NDVI Images for Wet and Dry Seasons </b> </font> 

<font face="Calibri" size="3">Now we can pick the bands and plot the Sentinel-1 and Landsat NDVI images of the subset. Change the band numbers to the bands we are interested in. </font>

<br>
<font face="Calibri" size="4"> <b> 6.1 Dry Season Plot</b> </font> 

In [ ]:
# DRY SEASON PLOT
bandnbrs=(30,5,42)
ebd_plot(bandnbrs)

<br>
<font face="Calibri" size="4"> <b> 6.2 Wet Season Plot</b> </font> 

In [ ]:
# Wet SEASON PLOT
bandnbrs=(52,20,53)
ebd_plot(bandnbrs)

<font face="Calibri" size="3"> In the figure above, for bands 52 Sentinel-1 VV, 20 Sentinel-1 VH and 53 NDVI aquired a few days earlier, see the inverse relationship. Where Sentinel-1 exhibits low backscatter, NDVI shows relatively higher NDVI. <b>What are the reasons for this in this environment?</b></font>

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u> </font></b> 

<font face="Calibri" size="3"> Pick different bands to compare. Look at the list of the dates for SAR data and Landsat Data acquisitions at the above. For example, compare bands from the dry and wet seasons 2016.
</font>
</div>
<br>
<hr>

<br>
<hr>
<font face="Calibri" size="5"> <b> 7. Compare the C-Band Radar Forest Degradation Index (RFDI) to the Landsat-8 NDVI </b> </font> 

<img src="NotebookAddons/RFDI.JPG" style="padding:5px;" width="430" align="right" /></font><font face="Calibri" size="3"> According to Lecture 7, we can compute a Radar Forest Degradation Index (RFDI) from polarimetric SAR data. The true definition of the RFDI is as follows:
<br> <br>
$RFDI = \frac{\gamma^0_{HH} - \gamma^0_{HV}}{\gamma^0_{HH} + \gamma^0_{HV}}$
<br><br>
Here, the terms are all radiometrically corrected imagery. The value of <i>RFDI</i> varies between 0 and 1. In general, RFDI can be used to detect both loss of forest cover and its recovery after a disturbance. It is best applied to L-band SAR imagery but can be applied to C-band as well. Approximate guidelines for the interpretation of the <i>RFDI</i> are shown on your right.

As you see from the equation above, the <i>RFDI</i> in its origial form requires <i>HH</i> and <i>HV</i> channels. For Sentinel-1, these bands are often not available (default mode over land is <i>VV</i> anv <i>VH</i>. Hence, we will work with a slightly modified version of the <i>RFDI</i> according to:
<br> <br>
$RFDI_{S1} = \frac{\gamma^0_{VV} - \gamma^0_{VH}}{\gamma^0_{VV} + \gamma^0_{VH}}$
<br>
</font>

<hr>
<font face="Calibri" size="3"> In the following code cell, we will <b>plot the <i>RFDI</i> for selected bands</b>. Please make sure to <b>pick same-date VV and VH images</b> to plot the RFDI and a <b>simliar-date Landsat-8 scene</b> for comparison.

<font color='rgba(200,0,0,0.2)'><b>Note:</b> The <i>RFDI</i> often appears a bit noisy. I am applying a $3\times3$ spatial filter to the data to reduce the noise. Hence, <b>the creation of the figure may take a short while. Please be patient.</b></font>
</font>

In [ ]:
# Plot RFDI vs NDVI
bandnbrs=(29,3,41)
RFDI_NDVI_plot(bandnbrs)

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u> </font></b> 

<font face="Calibri" size="3"> Please assess the information shown in the figure. While analyzing the figure, please consider the following questions:
- How does the RFDI compare to the NDVI for the bands you picked?
- Pick different bands and continue comparing RFDI and NDVI.
- What might be reasons for the differences you see?
</font>
</div>
<br>
<hr>

<br>
<hr>
<font face="Calibri" size="5"> <b> 8. Time Series Profiles of  C-Band Backscatter and NDVI </b> </font> 

<font face="Calibri" size="3"> We compute the image means of each time step in the time series stack and plot them together. </font>

<br>
<font face="Calibri" size="4"> <b> 8.1 Prepare Sentinel-1 Data Stack </b> </font> 

In [ ]:
caldB=-83
calPwr = np.power(10.,caldB/10.)
meanim = np.power(img_handle[0].GetRasterBand(bandnbrs[0]).ReadAsArray(*subset_sentinel),2.)*calPwr
s_ts=[]
for idx in (0,1):
    means=[]
    for i in range(bands[idx]):
        rs=img_handle[idx].GetRasterBand(i+1).ReadAsArray(*subset_sentinel)
        # 1. Conversion to Power
        rs_pwr=np.power(rs,2.)*calPwr
        meanim = (meanim+rs_pwr)/2
        rs_means_pwr = np.mean(rs_pwr)
        rs_means_dB = 10.*np.log10(rs_means_pwr)
        means.append(rs_means_dB)
    s_ts.append(pd.Series(means,index=stindex[idx]))

<br>
<font face="Calibri" size="4"> <b> 8.2 Prepare Landsat-8 NDVI Data Stack </b> </font> 

In [ ]:
means=[]
idx=2
for i in range(bands[idx]):
    r=img_handle[idx].GetRasterBand(i+1).ReadAsArray(*subset_landsat)
    means.append(r[r!=-9999].mean()/10000.)
l_ts=pd.Series(means,index=ltindex)

<br>
<font face="Calibri" size="4"> <b> 8.3 Joint Plot of SAR Backscatter and NDVI of Image Subset  Means </b> </font> 

<font face="Calibri" size="3"> Now we plot the time series of the SAR backscatter and NDVI values scaled to the same time axis. We also show the time stamps for the images we display above. </font>

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(13,7))
# ax1.plot(s_ts.index,s_ts.values, 'r-')
s_ts[0].plot(ax=ax[0],color='red',label='C-VV',xlim=(min(min(ltindex),min(stindex[0])),
                                                     max(max(ltindex),max(stindex[0]))))
s_ts[1].plot(ax=ax[0],color='blue',label='C-VH')
ax[0].set_xlabel('Date')
ax[0].set_ylabel('Sentinel-1 $\gamma^o$ [dB]')

# Make the y-axis label, ticks and tick labels match the line color. ax1.set_ylabel('exp', color='b')
# ax1.tick_params('y', colors='b')
# ax[1] = ax1.twinx()
# s_ts.plot(ax=ax[1],share=ax[0])
l_ts.plot(ax=ax[1],sharex=ax[0],label='NDVI',xlim=(min(min(ltindex),min(stindex[0])),
                                                     max(max(ltindex),max(stindex[0]))),ylim=(0,0.75))
# ax[1].plot(l_ts.index,l_ts.values,color='green',label='NDVI')
ax[1].set_ylabel('NDVI')
ax[0].set_title('Sentinel-1 Backscatter')
ax[1].set_title('Landsat NDVI')

ax[0].axvline(stindex[0][bandnbrs[0]-1],color='cyan',label='Sent. Date')
ax[1].axvline(ltindex[bandnbrs[2]-1],color='green',label='NDVI Date')
_=fig.legend(loc='center right')
_=fig.suptitle('Time Series Profiles of Sentinel-1 SAR Backscatter and Landsat-8 NDVI ')
# fig.tight_layout() 

<br>
<font face="Calibri" size="4"> <b> 8.4 Comparison of Time Series Profiles at Point Locations </b> </font> 

<font face="Calibri" size="3"> We will pick a pixel location in the SAR image, find the corresponding location in the Landsat NDVI stack and plot the joint time series.

To aid in the identification of pixel locations of interest, we will <b>first plot an average SAR image below</b>. Look at the bottom right of the figure to find line/pixel information for your cursor location. Use the cursor and the display on the bottom right to identify points you want to inspect.</font>

In [ ]:
def gray_plot(image, vmin=0, vmax=2, return_ax=False):
    '''Plots an image in grayscale.
    
    Parameters:
    - image: 2D array of raster values
    - vmin: Minimum value for colormap
    - vmax: Maximum value for colormap
    - return_ax: Option to return plot axis

    '''
    ax = plt.imshow(image, cmap = plt.cm.gist_gray)
    plt.clim(vmin,vmax)
    if return_ax:
        return(ax)
    
###############################################################################

def big_fig(x=20,y=10):
    '''Initializes a large figure.
    
    Parameters:
    - x, y: X and Y figure dimensions
    '''
    return(plt.figure(figsize=(x,y)))

In [ ]:
%matplotlib notebook
class pixelPicker:
    def __init__(self, image, width, height):
        self.x = None
        self.y = None
        self.fig = plt.figure(figsize=(width, height))
        self.ax = self.fig.add_subplot(111, visible=False)
        self.rect = patches.Rectangle(
            (0.0, 0.0), width, height, 
            fill=False, clip_on=False, visible=False)      
        self.rect_patch = self.ax.add_patch(self.rect)
        self.cid = self.rect_patch.figure.canvas.mpl_connect('button_press_event', 
                                                             self)
        self.image = image
        self.plot = self.gray_plot(self.image, fig=self.fig, return_ax=True)
        self.plot.set_title('Select a Point of Interest')
        
        
    def gray_plot(self, image, vmin=None, vmax=None, fig=None, return_ax=False):
        '''
        Plots an image in grayscale.
        Parameters:
        - image: 2D array of raster values
        - vmin: Minimum value for colormap
        - vmax: Maximum value for colormap
        - return_ax: Option to return plot axis
        '''
        if vmin is None:
            vmin = np.nanpercentile(self.image, 1)
        if vmax is None:
            vmax = np.nanpercentile(self.image, 99)
        ax = fig.add_axes([0.1,0.1,0.8,0.8])
        ax.imshow(image, cmap=plt.cm.gist_gray, vmin=vmin, vmax=vmax)
        if return_ax:
            return(ax)
        
    
    def __call__(self, event):
        print('click', event)
        self.x = event.xdata
        self.y = event.ydata
        for pnt in self.plot.get_lines():
            pnt.remove()
        plt.plot(self.x, self.y, 'ro')

In [ ]:
fig_xsize = 9.5
fig_ysize = 9.5
my_plot = pixelPicker(10.*np.log10(meanim), fig_xsize, fig_ysize)

In [ ]:
sarloc = (ceil(my_plot.x), ceil(my_plot.y))
print(sarloc)

<font face="Calibri" size="3"> Now we use the geotrans info to find the same location in the Landsat image. </font>

In [ ]:
ref_x=geotrans[0][0]+sarloc[0]*geotrans[0][1]
ref_y=geotrans[0][3]+sarloc[1]*geotrans[0][5]
print('UTM Coordinates      ',ref_x,ref_y)
print('SAR pixel/line       ',sarloc[0],sarloc[1])
target_pixel=round((ref_x-geotrans[2][0])/geotrans[2][1])
target_line=round((ref_y-geotrans[2][3])/geotrans[2][5])
print('Landsat pixel/line   ',target_pixel,target_line)
subset_sentinel=(sarloc[0],sarloc[1],1,1)
subset_landsat=(target_pixel,target_line,1,1)

<font face="Calibri" size="3"> Read the image data at these locations. </font>

In [ ]:
caldB=-83
calPwr = np.power(10.,caldB/10.)

s_ts=[]
for idx in (0,1):
    means=[]
    for i in range(bands[idx]):
        rs=img_handle[idx].GetRasterBand(i+1).ReadAsArray(*subset_sentinel)
        # 1. Conversion to Power
        rs_pwr=np.power(rs,2.)*calPwr
        rs_means_pwr = np.mean(rs_pwr)
        rs_means_dB = 10.*np.log10(rs_means_pwr)
        means.append(rs_means_dB)
    s_ts.append(pd.Series(means,index=stindex[idx]))
    
means=[]
idx=2
for i in range(bands[idx]):
    r=img_handle[idx].GetRasterBand(i+1).ReadAsArray(*subset_landsat)
    means.append(r[r!=-9999].mean()/10000.)
l_ts=pd.Series(means,index=ltindex)

<font face="Calibri" size="3"> Plot the joint time series. </font>

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(2,1,figsize=(13,7))
# ax1.plot(s_ts.index,s_ts.values, 'r-')
s_ts[0].plot(ax=ax[0],color='red',label='C-VV',xlim=(min(min(ltindex),min(stindex[0])),
                                                     max(max(ltindex),max(stindex[0]))))
s_ts[1].plot(ax=ax[0],color='blue',label='C-VH')
ax[0].set_xlabel('Date')
ax[0].set_ylabel('Sentinel-1 $\gamma^o$ [dB]')

# Make the y-axis label, ticks and tick labels match the line color. ax1.set_ylabel('exp', color='b')
# ax1.tick_params('y', colors='b')
# ax[1] = ax1.twinx()
# s_ts.plot(ax=ax[1],share=ax[0])
l_ts.plot(ax=ax[1],sharex=ax[0],label='NDVI',xlim=(min(min(ltindex),min(stindex[0])),
                                                     max(max(ltindex),max(stindex[0]))),ylim=(0,0.75))
# ax[1].plot(l_ts.index,l_ts.values,color='green',label='NDVI')
ax[1].set_ylabel('NDVI')
ax[0].set_title('Sentinel-1 Backscatter')
ax[1].set_title('Landsat NDVI')

ax[0].axvline(stindex[0][bandnbrs[0]-1],color='cyan',label='Sent. Date')
ax[1].axvline(ltindex[bandnbrs[2]-1],color='green',label='NDVI Date')
_=fig.legend(loc='center right')
_=fig.suptitle('Time Series Profiles of Sentinel-1 SAR Backscatter and Landsat-8 NDVI ')
# fig.tight_layout() 

<font face="Calibri" size="3"> Interpret these time series profiles. While generally the seasonal trends are visible in both like (VV) and cross-polarized (VH) data, and correlate well with the NDVI temporal profile, the cross-polarized response is less pronounced at many pixel locations. </font> 

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u> </font></b> 

<font face="Calibri" size="3"> Analyze different pixel locations and replot the figure above. Interpret the result with respect to forest, non-forest, deforestation and forest degradation signatures. In your interpretation look for image signals of strong rain events in the SAR data and cloud covered scenes in the Landsat imagery.
</font>
</div>
<br>
<hr>

<font face="Calibri" size="2"> <i>Exercise9-SAROpticalComparison.ipynb - Version 1.3.0 - April 2021 </i>
    <br>
        <b>Version Changes:</b>
    <ul>
        <li>from osgeo import gdal</li>
        <li>namespace asf_notebook</li>
        <li>%matplotlib inline and %matplotlib notebook in correct locations</li>
    </ul>
    </i>
</font>